In [1]:
# imports

#!pip install watermark
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import itertools

In [2]:
# get stop words

stop_words = stopwords.words('german')
print(stop_words[:10]) # example of stop words

['aber', 'alle', 'allem', 'allen', 'aller', 'alles', 'als', 'also', 'am', 'an']


In [3]:
filename1 = "processed_articles_2020-01-01_2022-06-30_1.csv"
filename2 = "processed_articles_2020-01-01_2022-06-30_2.csv"
articles1 = pd.read_csv(filename1)
articles2 = pd.read_csv(filename2)

articles = pd.concat([articles1, articles2], ignore_index=True).drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])


articles

,title,url,content,time
0,Rückzug von Prinz Harry und Meghan: Flucht ode...,https://www.faz.net/aktuell/gesellschaft/mensc...,Queen versuchen Treffen Sandringham offen Bruc...,2020-01-14 08:00:00
1,Zehntausende Iraner bei Trauerfeiern für Qasse...,https://www.faz.net/aktuell/politik/ausland/ze...,Zehntausend Iraner Trauerzug amerikanisch Rake...,2020-01-05 08:00:00
2,Von der Leyen in Davos: Globale Kooperation un...,https://www.faz.net/aktuell/wirtschaft/weltwir...,Europa suchen Rolle wandelnd Welt Kommissionsp...,2020-01-22 08:00:00
3,Coronavirus aus China ist von Mensch zu Mensch...,https://www.faz.net/aktuell/gesellschaft/gesun...,neu Erkenntnis Bezug Lungenkrankheit dürfen An...,2020-01-20 08:00:00
4,Flugzeugabsturz in Iran: Vieles deutet auf ein...,https://www.faz.net/aktuell/politik/ausland/fl...,unabhängig Untersuchungsbericht dürfen bald ge...,2020-01-10 08:00:00
...,...,...,...,...
11253,"Drei Gründe, warum in Italien der Populismus i...",https://www.nzz.ch/international/lega-und-cinq...,Lega Cinque Stelle Kommunalwahl Italien misera...,2022-06-29 07:00:00
11254,EU-Kandidaten-Status für Ukraine und Moldau al...,https://www.nzz.ch/pro-global/mit-dem-versprec...,Versprechen Ukraine Moldau positionieren Eu ge...,2022-06-29 07:00:00
11255,USA Wahlen: Joe Bidens hohes Alter erschreckt ...,https://www.nzz.ch/international/usa-wahlen-jo...,Joe bidens hoch Alter erschrecken Demokrat alt...,2022-06-29 07:00:00
11256,Busspur-Abbau in Zürich? Stadtrat Michael Baum...,https://www.nzz.ch/zuerich/busspur-abbau-in-zu...,Busspurabbau Zürich Oberst Trämler Michael Bau...,2022-06-29 07:00:00


In [4]:
# Starting with the CountVectorizer/TfidfTransformer approach...


#cvec = CountVectorizer(stop_words=stop_words, min_df=0.005, max_df=.1, ngram_range=(1,2))
cvec = TfidfVectorizer(analyzer='word', ngram_range=(1, 1), min_df = 0.005, stop_words=stop_words)
#tfidf_matrix = tf.fit_transform(data['title'])

#cvec

In [5]:
# Calculate all the n-grams found in all documents
cvec.fit(articles['content'])
tfidf_matrix = cvec.fit_transform(articles['content'])
list(itertools.islice(cvec.vocabulary_.items(), 20))
tfidf_matrix

<11258x6619 sparse matrix of type '<class 'numpy.float64'>'
	with 2020967 stored elements in Compressed Sparse Row format>

In [6]:
# Check how many total n-grams we have
len(cvec.vocabulary_)

6619

In [7]:
phrase = "Eintracht Frankfurt wird den Europapokal gewinnen"

In [8]:
matrix2 = cvec.transform([phrase])
matrix2


<1x6619 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [9]:
similarity = pd.DataFrame(cosine_similarity(tfidf_matrix, matrix2), columns=['score'])
#pd.set_option('display.max_rows', None)
#pd.set_option('display.width', 2000)
similarity = similarity.merge(articles, how='outer', left_index=True, right_index=True)
similarity.sort_values(by=['score'], ascending=False)

,score,title,url,content,time
11063,0.545115,Eintracht Frankfurt: Ein Klub verblüfft Fussba...,https://www.nzz.ch/sport/eintracht-frankfurt-f...,wissen nie kommen Eintracht Frankfurt faszinie...,2022-05-19 07:00:00
11062,0.532895,Eintracht Frankfurt: Ein Klub will zurück ins ...,https://www.nzz.ch/sport/ein-trainer-und-seine...,Trainer Mission Oliver Glasner Eintracht Frank...,2022-05-19 07:00:00
2177,0.490228,Bundesliga: Früherer Eintracht-Torjäger Bernd ...,https://www.faz.net/aktuell/sport/rhein-main-s...,Mann Phänomen egal Training Spiel Ball Bernd N...,2021-10-27 07:00:00
1305,0.485790,Eintracht Frankfurt: Sportdirektor Bruno Hübne...,https://www.faz.net/aktuell/sport/fussball/bun...,Bruno Hübner Auslaufen Vertrag Saisonende Dien...,2021-02-17 08:00:00
11061,0.474362,Wegen Siegesfeier von Eintracht Frankfurt: Hoc...,https://www.nzz.ch/international/wegen-der-sie...,wegen Siegesfeier Eintracht Frankfurt Hochzeit...,2022-05-19 07:00:00
...,...,...,...,...,...
4149,0.000000,Nordkorea nimmt nicht an Olympischen Spielen teil,https://www.dw.com/de/nordkorea-nimmt-nicht-an...,Nordkorea Teilnahme olympisch Sommerspiel Toki...,2021-04-06 07:00:00
4150,0.000000,WWF: EU ist einer der größten Waldzerstörer,https://www.dw.com/de/wwf-eu-ist-einer-der-gr%...,Soja rindfleisch Kaffee Kakao Kunde europäisch...,2021-04-13 07:00:00
4151,0.000000,"Politiker: Nawalny wird ""gezielt gefoltert""",https://www.dw.com/de/politiker-nawalny-wird-g...,Brief russisch Oppositionspolitiker Alexej Naw...,2021-04-10 07:00:00
4152,0.000000,Corona überrollt die Menschenrechte,https://www.dw.com/de/corona-%C3%BCberrollt-di...,Menschenrechtslage Coronapandemie rund Globus ...,2021-04-06 07:00:00


In [10]:
#%load_ext watermark
#%watermark -v -m -p numpy,nltk,pandas,itertools,sklearn